<a href="https://colab.research.google.com/github/peterbmob/DHMVADoE/blob/main/Excercises/fractional_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Fractional 2$^k$  Factorial Designs
Motivation
The prior section showed an example of what an experimental design might like look like for 6 variables. However, this resulted in a  2$^6$=64
  experiment design campaign. This is potentially a major issue - if my experiments take 6 hours, and have to be staggered over working hours on weekdays, you're looking at almost 90 days turnaround time, assuming each experiment is carried out flawlessly. This is simply not a realistic view of experimentation.

In addition, we saw that a three-coefficient model captured nearly as much detail as a 64-coefficient model. By reducing the number of input variables we looked at, we turned certain experiments into replicates (because the only thing changed bewteen them were insignificant variables or variable combinations).

But we can halve or quarter our effort, and substantially improve our effectiveness in the lab, by carefully selecting experiments at each stage of the experiment to reveal a maximum amount of information, and avoiding as much as possible these kinds of duplicate experiments, through a fractional factorial design.

In [ ]:
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import pylab

import scipy.stats as stats
import statsmodels.api as sm

After re-casting the problem in a general form, we begin with the experimental design matrix. If we were to construct the full factorial for our  2$^6$ factorial example, we would again have 64 rows in our experimental design matrix dataframe, corresponding to 64 experiments to run.

In [ ]:
column_labs = ['x%d'%(i+1) for i in range(6)]
encoded_inputs = list( itertools.product([-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1]) )

# Create the experiment design table (same as the book):
doe=pd.DataFrame(encoded_inputs)
doe=doe[doe.columns[::-1]]
doe.columns=['x%d'%(i+1) for i in range(6)]

print(len(doe))

## Design Matrix
Let's talk a bit more about the design matrix. Each column of the design matrix corresponds to a unique coded input variable value (−1,+1). But each experiment also has a corresponding coded value for each two-variable interaction  x$_i$,x$_j$, and for each three-variable interaction  x$_k$,x$_m$,x$_n$, and so on.

These interactions are simply the product of each coded variable value. For example, if

\begin{equation}
x_1=−1 \\
\end{equation}

\begin{equation}
x_2=+1 \\
\end{equation}

\begin{equation}
x_3=+1
\end{equation}

then two-variable interaction effects can be computed as:

\begin{equation}
x_{12}=−1\times+1 = -1 \\
\end{equation}

\begin{equation}
x_{13}=−1\times+1 = −1 \\
\end{equation}

\begin{equation}
x_{23}=+1\times+1 = +1 \\
\end{equation}

and three-variable interaction effects are:

\begin{equation}
x_{123}=−1\times−1\times+1=+1
\end{equation}

Now we can add new columns to our experimental design matrix dataframe, representing coded values for higher-order interaction effects:

In [ ]:
doe['x1-x2-x3-x4'] = doe.apply( lambda z : z['x1']*z['x2']*z['x3']*z['x4'] , axis=1)
doe['x4-x5-x6']    = doe.apply( lambda z : z['x4']*z['x5']*z['x6'] , axis=1)
doe['x2-x4-x5']    = doe.apply( lambda z : z['x2']*z['x4']*z['x5'] , axis=1)

doe[0:10]

The multi-variable columns can be used to fractionate our design.

## Half Factorial
Suppose we pick a high-order interaction effect at random - e.g., x$_1\times$x$_2\times$x$_3\times$x$_4$ - and assume it will be unimportant. Our assumption allows us to cut out any experiments that are intended to give us information about the effect of  x$_1$x$_2$x$_3$x$_4$.

For any two groups of experiments, if one group has
\begin{equation}
x_1x_2x_3x_4=+1
\end{equation}

and the other group has
\begin{equation}
x_1x_2x_3x_4=−1
\end{equation}

then based on our assumption that that interaction effect will be unimportant, one of those two groups can be thrown out.

Fortunately, the first time a variable is eliminated, no matter which variable it is, the number of experiments is cut in half. Further eliminations of variables continue to cut the number of experiments in half. So a six-factor experimental design could be whittled down as follows:

Six-factor, two-level experiment design:

- n=2,  k=6,  2$^6$ experimental design
- Full factorial:  2$^6$=64 experiments
- Half factorial:  2$^{6−1}$=32 experiments
- $\frac{1}{4}$ Fractional factorial:  2$^{6−2}$=16 experiments
- $\frac{1}{8}$ Fractional factorial:  2$^{6−3}$=8  experiments
- $\frac{1}{16}$ Fractional factorial:  2$^{6−4}$=4  experiments

In general, for an **n$^k$ experiment design** (n factor, k level), a **$\frac{1}{2p}$ fractional factorial** can be defined as:
- $\frac{1}{2p}$ Fractional factorial:  2$^{n−p}$  experiments

Note that as the fractional factorial gets narrower, and the experiments get fewer, the number of aliased interaction effects gets larger, until not even interaction effects can be distinguished, but only main effects. (Screening designs, such as Plackett-Burman designs, are based on this idea of highly-fractionated experiment design.)

For now, let's look at the half factorial: 32 experiments, with the reduction in variables coming from aliasing the interaction effect  x$_1$x$_2$x$_3$x$_4$:

In [ ]:
print(len( doe[doe['x1-x2-x3-x4']==1] ))

## Costs and Benefits
The benefits are obvious - we've halved the number of experiments our experiment design requires. But at what cost?

The first 32 experiments, where  x$_1$x$_2$x$_3$x$_4$=+1, give us information at a positive level of that input variable combination. To get information at a negative level of that input variable combination (i.e.,  x$_1$x$_2$x$_3$x$_4$=−1
 ), we need 32 additional experiments.

Our assumption is that changing x$_1$x$_2$x$_3$x$_4$ from high to low will have no effect on the observable y.

This also modifies the information we get about higher-order interaction effects. For example, we've assumed:
\begin{equation}
x_1x_2x_3x_4=+1
\end{equation}

We can use this identity to figure out what information we're missing when we cut out the 32 experiments. Our assumption about the fourth-order interaction also changes fifth- and sixth-order interactions:

\begin{equation}
(x_1x_2x_3x_4)=(+1) \\
\end{equation}

\begin{equation}
(x_1x_2x_3x_4)x_5=(+1)x_5 \\
\end{equation}

\begin{equation}
x_1x_2x_3x_4x_5=x_5
\end{equation}

meaning the fifth-order interaction effect x$_1$x$_2$x$_3$x$_4$x$_5$ has been aliased with the first-order main effect x$_5$. This is a safe assumption since it is extremely unlikely that a fifth-order interaction effect could be confounded with a first-order main effect. We can derive other relations, using the fact that any factor squared is equivalent to (+1), so that:

\begin{equation}
(x_1x_2x_3x_4)=+1 \\
\end{equation}

\begin{equation}
(x_1x_2x_3x_4)x_1=(+1)x_1 \\
\end{equation}

\begin{equation}
(x^2_1x_2x_3x_4)=(+1)x_1 \\
\end{equation}

\begin{equation}
x_2x_3x_4=x_1
\end{equation}

The sequence of variables selected as the interaction effect to be used as the experimental design basis is called the generator, and is denoted $I$:

\begin{equation}
I=x_1x_2x_3x_4
\end{equation}

and we set $I$=+1 or $I$=−1.

In [ ]:
# Defining multiple DOE matrices:

# DOE 1 based on identity I = x1 x2 x3 x4
doe1 = doe[doe['x1-x2-x3-x4']==1]

# DOE 2 based on identity I = x4 x5 x6
doe2 = doe[doe['x4-x5-x6']==-1]

# DOE 3 based on identity I = x2 x4 x5
doe3 = doe[doe['x2-x4-x5']==-1]

In [ ]:
doe1[column_labs].T

In [ ]:
doe2[column_labs].T

In [ ]:
doe3[column_labs].T

Each of the dataframes above represents a different fractional factorial design.

## $\frac{1}{4}$ Fractional Designs
To further reduce the number of experiments, two identities can be used. The number of experiments is cut in half for each identity. We already have one identity,
\begin{equation}
I=x_1x_2x_3x_4=+1
\end{equation}

now let's define another one:
\begin{equation}
I_2=x_4x_5x_6=1
\end{equation}

Our resulting factorial matrix can be reduced the same way. In Python, we use the logical_and function to ensure our two conditions are satisfied.

In [ ]:
quarter_fractional_doe = doe[ np.logical_and( doe['x1-x2-x3-x4']==1, doe['x4-x5-x6']==1 ) ]
print("Number of experiments: %d"%(len(quarter_fractional_doe[column_labs])))
quarter_fractional_doe[column_labs].T

With the quarter-fractional factorial design, what information do we lose? We know already which interaction effects are aliased with main effects:

\begin{equation}
x_4x_5x_6=(+1) \\
\end{equation}

\begin{equation}
(x_4x_5x_6)x_1=(+1)x_1 \\
\end{equation}

\begin{equation}
x_1x_4x_5x_6=x_1 \\
\end{equation}

\begin{equation}
x_2x_4x_5x_6=x_2 \\
\end{equation}

\begin{equation}
x_3x_4x_5x_6=x_3
\end{equation}

We can use this information to design our experiments to cover particular interaction effects we know to be important, or ignore others we don't expect to be significant.

## Estimability and Aliasing

Any factorial effect in an alias string is only estimable if all other effects in that string are assumed zero38. Wd can study this further by introducing the alias matrix.

**Definition** Assume a linear data generating model
$$
\begin{equation}
y = X_1β_1 + X_2β_2 + ε,
\end{equation}
$$
where y is an n-vector of responses, $X_1$ and $X_2$ are n×p$_1$ and n×p$_2$ model matrices,respectively, with $β_1$ and $β_2$ corresponding  p$_1$- and p$_2$-vectors of parameters and random errors ε $N(0,I_nσ^2$).

If the submodel
$$
\begin{equation}
y = X_1β_1 + ε,
\end{equation}
$$
is fitted to the response data, then  
$$
\begin{equation}
\hat{\beta}_1 =(X^T_1X_1)^{-1}X^T_1y,
\end{equation}
$$
and
$$
\begin{equation}
E(\hat{\beta}_1) =β_1+(X^T_1X_1)^{-1}X^T_1X_2β_2=β_1+Aβ_2,
\end{equation}
$$
where
$$
\begin{equation}
A=(X^T_1X_1)^{-1}X^T_1X_2
\end{equation}
$$
is the alias matrix.

We also introduce an alternative definition of estimability.

**Definition** A linear combination of parameters $c^Tθ$ is estimable if and only if there exists a linear combination of the responses $a^Ty$ such that
$$
\begin{equation}
E(a^Ty)=c^Tθ.
\end{equation}
$$

Now assume that using a two-level fractional factorial design, we will estimate one factorial effect (equivalently, the corresponding regression coefficient) from each alias string. Then the **A** matrix will have entries 0, -1 or +1, depending on the defining relation of the fraction. Each regression parameter will be biased by the parameters corresponding to other factorial effects in the alias string. Hence, by the **Definition**, each factorial effect is only estimable under the assumption that all other factorial effects in the alias string are zero.

The Dexpy package allows for analysis of aliasing.

In [ ]:
!pip install dexpy

In [ ]:
import dexpy.factorial
design = dexpy.factorial.build_factorial(4, 8)
aliases, alias_coefs = dexpy.alias.alias_list("(X1+X2+X3+X4)**2", design)
print(aliases)

# Other designs
https://github.com/danieleongari/awesome-design-of-experiments lists many different DOE packages in python.
[DOEPY](https://doepy.readthedocs.io/en/latest/) and [pydoe](https://pypi.org/project/pyDOE/) have many other experimental designs.  

Install doepy using pip

In [ ]:
!pip install doepy

## Load the moodules we need

In [ ]:
from doepy import read_write, build

### Full factorial design
Let’s say you have a design problem with the following table for the parameters range. Imagine this as a generic example of a checmical process in a manufacturing plant. You have 3 levels of Pressure, 3 levels of Temperature, 2 levels of FlowRate, and 2 levels of Time.

In [ ]:
build.full_fact(
{'Pressure':[40,55,70],
'Temperature':[290, 320, 350],
'Flow rate':[0.2,0.4],
'Time':[5,8]}
)

### Latin Hypercube design
Sometimes, a set of randomized design points within a given range could be attractive for the experimenter to asses the impact of the process variables on the output. Monte Carlo simulations are close example of this approach.

However, a Latin Hypercube design is better choice for experimental design rather than building a complete random matrix as it tries to subdivide the sample space in smaller cells and choose only one element out of each subcell. This way, a more uniform spreading’ of the random sample points can be obtained.

User can choose the density of sample points. For example, if we choose to generate a Latin Hypercube of 12 experiments from the same input files, that could look like,

In [ ]:
build.space_filling_lhs(
{'Pressure':[40,55,70],
'Temperature':[290, 320, 350],
'Flow rate':[0.2,0.4],
'Time':[5,11]},
num_samples = 12
)

Of course, there is no guarantee that you will get the same matrix if you run this function because this are randomly sampled, but you get the idea!

Other functions to try on Try any one of the following designs,

- Full factorial: build.full_fact()
- 2-level fractional factorial: build.frac_fact_res()
- Plackett-Burman: build.plackett_burman()
- Sukharev grid: build.sukharev()
- Box-Behnken: build.box_behnken()
- Box-Wilson (Central-composite) with center-faced option: build.central_composite() with face='ccf' option
- Box-Wilson (Central-composite) with center-inscribed option: build.central_composite() with face='cci' option
- Box-Wilson (Central-composite) with center-circumscribed option: build.central_composite() with face='ccc' option
- Latin hypercube (simple): build.lhs()
- Latin hypercube (space-filling): build.space_filling_lhs()
- Random k-means cluster: build.random_k_means()
- Maximin reconstruction: build.maximin()
- Halton sequence based: build.halton()
- Uniform random matrix: build.uniform_random()

After perfroming the experiments, we analyze in the way we done in previous tutorials.